In [1]:
#Saturated Oxygen Data Correction

In [5]:
#This is to import libraries of different functions that allows the program do what it's doing
%pylab inline
from netCDF4 import Dataset
import pandas as pd
from pandas import Series , DataFrame
import csv
from IPython.core.display import Image
import urllib2
import StringIO
from thredds_crawler.crawl import Crawl
from seawater.extras import satO2

Populating the interactive namespace from numpy and matplotlib


In [6]:
data = urllib2.urlopen("http://argo.jcommops.org/FTPRoot/Argo/Status/argo_all.txt").readlines()
data[1].replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '')
#So this works for a specific row but not the whole thing O.o
#Make a loop that runs through all of the rows one by one (since all together doesn't work)

'6900877,99731,ARGOS,5058,5058,29/06/2014,17/04/2014,06,2014,57.5161,-21.8886,POSEIDON,P741,21/07/2015,59.9372,-22.6524,APEX,"Giese, Holger",holger.giese@bsh.de,Argo BSH,GERMANY,386.3628,0.86,1,78,0,0,0,120,1000,1200,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0'

In [8]:
d2 = [d.replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '') for d in data]
d2[:5]

['WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,LON0,SHIP,CRUISE,DATE_,LAT_,LON_,MODEL,FULL_NAME,EMAIL,PROGRAM,COUNTRY,AGE,AGE_WEIGHT,GTS,PROFILES,DMPROFILES,GTSPRESS,GREYLIST,CYCLE_TIME,DRIFT_PRESS,PROFILE_PRESS,OXYGEN,FLUO,TURBIDITY,RAFOS,CDOM,BEAM,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED',
 '6900877,99731,ARGOS,5058,5058,29/06/2014,17/04/2014,06,2014,57.5161,-21.8886,POSEIDON,P741,21/07/2015,59.9372,-22.6524,APEX,"Giese, Holger",holger.giese@bsh.de,Argo BSH,GERMANY,386.3628,0.86,1,78,0,0,0,120,1000,1200,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0',
 '6900876,99730,ARGOS,5057,5057,29/06/2014,17/04/2014,06,2014,56.9983,-21.83,POSEIDON,P471,20/07/2015,59.2584,-10.8677,APEX,"Giese, Holger",holger.giese@bsh.de,Argo BSH,GERMANY,386.1254,0.86,1,78,0,0,0,120,1000,1200,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0',
 '6902554,139249,ARGOS,OIN14DAR03,OIN14DAR03,23/06/2014,21/05/2014,06,2014,41.0021,-20.2838,POSEIDON,471,20/07/2015,37.0153,-26.1956,ARVOR,"Giese, Holg

In [9]:
#Latest attempt
df = pd.DataFrame(d2)
type(df)

pandas.core.frame.DataFrame

In [10]:
with open('d2.csv', 'w') as f:
    for row in d2:
        f.write(row + '\n')

df = pd.read_csv('d2.csv')
df[:5]

,WMO,TELECOM,TTYPE,MY_ID,SERIAL_NO,DATE0,NOTIF_DATE,MONTH0,YEAR0,LAT0,...,BACKSCATT,DOWNWELL,SST,SSS,PAL,OTHER,ICE_DETECT,ACTIVE,BEACHED,RETIRED
0,6900877,99731,ARGOS,5058,5058,29/06/2014,17/04/2014,6,2014,57.5161,...,0,0,0,0,0,0,0,1,0,0
1,6900876,99730,ARGOS,5057,5057,29/06/2014,17/04/2014,6,2014,56.9983,...,0,0,0,0,0,0,0,1,0,0
2,6902554,139249,ARGOS,OIN14DAR03,OIN14DAR03,23/06/2014,21/05/2014,6,2014,41.0021,...,0,0,0,0,0,0,0,1,0,0
3,6902553,139248,ARGOS,OIN14DAR02,OIN14DAR02,22/06/2014,21/05/2014,6,2014,38.9999,...,0,0,0,0,0,0,0,1,0,0
4,6902552,139247,ARGOS,OIN14DAR01,OIN14DAR01,21/06/2014,21/05/2014,6,2014,37.0001,...,0,0,0,0,0,0,0,1,0,0


In [11]:
#This prints out certain columns (of my choosing) with all the rows still present
fd_oga = df.loc[: , ['WMO' , 'OXYGEN' , 'GREYLIST' , 'AGE' , 'DATE_' , 'LAT_' , 'LON_'] ] # oga = oxygen, grey list, and age
fd_oga[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,386.3628,21/07/2015,59.9372,-22.6524
1,6900876,1,0,386.1254,20/07/2015,59.2584,-10.8677
2,6902554,0,0,392.3855,20/07/2015,37.0153,-26.1956
3,6902553,0,0,392.6277,19/07/2015,35.7544,-27.7932
4,6902552,0,0,392.5143,18/07/2015,34.8368,-18.4200


In [12]:
#This takes the fd_oga table and only shows the rows that have oxygen data
fd_oxygen = fd_oga.loc[ fd_oga.loc[: , 'OXYGEN'] == 1 , :]
fd_oxygen[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,386.3628,21/07/2015,59.9372,-22.6524
1,6900876,1,0,386.1254,20/07/2015,59.2584,-10.8677
40,1900943,1,0,317.3993,27/12/2007,20.6700,-31.6260
66,6902546,1,0,0.4931,21/01/2014,64.6896,0.0736
67,6902544,1,0,590.2862,29/06/2015,76.9700,-1.1120


In [13]:
#This takes the fd_oga table and only shows the rows that aren't grey listed     #gl = grey list
fd_gl = fd_oga.loc[ fd_oga.loc[: , 'GREYLIST'] == 0 , :] #gl = grey list
fd_gl[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,386.3628,21/07/2015,59.9372,-22.6524
1,6900876,1,0,386.1254,20/07/2015,59.2584,-10.8677
2,6902554,0,0,392.3855,20/07/2015,37.0153,-26.1956
3,6902553,0,0,392.6277,19/07/2015,35.7544,-27.7932
4,6902552,0,0,392.5143,18/07/2015,34.8368,-18.4200


In [14]:
#This takes the fd_oga table and only shows the rows that are at least 340 days old
fd_age = fd_oga.loc[ fd_oga.loc[: , 'AGE'] >= 340. , :]
fd_age[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,386.3628,21/07/2015,59.9372,-22.6524
1,6900876,1,0,386.1254,20/07/2015,59.2584,-10.8677
2,6902554,0,0,392.3855,20/07/2015,37.0153,-26.1956
3,6902553,0,0,392.6277,19/07/2015,35.7544,-27.7932
4,6902552,0,0,392.5143,18/07/2015,34.8368,-18.4200


In [15]:
#Merging the fd_oxygen table with the fd_gl table
fd_merge_og = pd.merge(fd_oxygen , fd_gl) #og = oxygen and grey list
fd_merge_og[:5]

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,386.3628,21/07/2015,59.9372,-22.6524
1,6900876,1,0,386.1254,20/07/2015,59.2584,-10.8677
2,1900943,1,0,317.3993,27/12/2007,20.6700,-31.6260
3,6902546,1,0,0.4931,21/01/2014,64.6896,0.0736
4,6902544,1,0,590.2862,29/06/2015,76.9700,-1.1120


In [35]:
#Later work will go off of this table

#FINALLY HAVE ONLY THE DATA I DESIRE
#PRAISE HELIX
#After all the trial and error (=^.^=)

fd_merge_oga = pd.merge(fd_merge_og , fd_age)
fd_merge_oga[:5]
#Merged the fd_merge_og table with the fd_age table

#Inactive -> active = 0

,WMO,OXYGEN,GREYLIST,AGE,DATE_,LAT_,LON_
0,6900877,1,0,386.3628,21/07/2015,59.9372,-22.6524
1,6900876,1,0,386.1254,20/07/2015,59.2584,-10.8677
2,6902544,1,0,590.2862,29/06/2015,76.9700,-1.1120
3,6902545,1,0,521.7191,28/06/2015,69.3350,5.0190
4,6902547,1,0,526.0354,02/07/2015,68.2500,9.4520


In [17]:
gd = urllib2.urlopen("ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt").readlines()
gd[:10]

['# Title : Metadata directory file of the Argo Global Data Assembly Center\n',
 '# Description : The directory file describes all metadata files of the argo GDAC ftp site.\n',
 '# Project : ARGO\n',
 '# Format version : 2.0\n',
 '# Date of update : 20150723202521\n',
 '# FTP root number 1 : ftp://ftp.ifremer.fr/ifremer/argo/dac\n',
 '# FTP root number 2 : ftp://usgodae.usgodae.org/pub/outgoing/argo/dac\n',
 '# GDAC node : CORIOLIS\n',
 'file,profiler_type,institution,date_update\n',
 'aoml/13857/13857_meta.nc,845,AO,20120521144513\n']

In [18]:
#May or may not even need this
gd = urllib2.urlopen("ftp://ftp.ifremer.fr/ifremer/argo/ar_index_global_meta.txt").readlines() #gd = global data

gd1 = [g.replace('\0' , '').replace('\xff' , '').replace('\xfe' , '').replace('\r\n' , '') for g in gd]

gd1[:10]
#Now to get it so when a user inputs a float number, the DAC of that number shows up

['# Title : Metadata directory file of the Argo Global Data Assembly Center\n',
 '# Description : The directory file describes all metadata files of the argo GDAC ftp site.\n',
 '# Project : ARGO\n',
 '# Format version : 2.0\n',
 '# Date of update : 20150723202521\n',
 '# FTP root number 1 : ftp://ftp.ifremer.fr/ifremer/argo/dac\n',
 '# FTP root number 2 : ftp://usgodae.usgodae.org/pub/outgoing/argo/dac\n',
 '# GDAC node : CORIOLIS\n',
 'file,profiler_type,institution,date_update\n',
 'aoml/13857/13857_meta.nc,845,AO,20120521144513\n']

In [19]:
with open('gd1.csv', 'w') as f:
    for row in gd1:
        f.write(row + '\n')

gd_table = pd.read_csv('gd1.csv' , comment = '#')
gd_table[:5]

,file,profiler_type,institution,date_update
0,aoml/13857/13857_meta.nc,845,AO,20120521144513
1,aoml/13858/13858_meta.nc,845,AO,20130222100319
2,aoml/13859/13859_meta.nc,845,AO,20120521144513
3,aoml/15819/15819_meta.nc,845,AO,20120521144513
4,aoml/15820/15820_meta.nc,845,AO,20120521144513


In [20]:
# Would like to read off Internet
gd_table.loc[:,['file']][:10]

,file
0,aoml/13857/13857_meta.nc
1,aoml/13858/13858_meta.nc
2,aoml/13859/13859_meta.nc
3,aoml/15819/15819_meta.nc
4,aoml/15820/15820_meta.nc
5,aoml/15851/15851_meta.nc
6,aoml/15852/15852_meta.nc
7,aoml/15853/15853_meta.nc
8,aoml/15854/15854_meta.nc
9,aoml/15855/15855_meta.nc


In [21]:
bList =[12,35,67,98]
for i in range(50):
    if i in bList:
        print i, "in bList"

12 in bList
35 in bList


In [50]:
good_floats = []
for index,row in fd_merge_oga.loc[:,['WMO']].iterrows():
    good_floats.append(row['WMO'])
print good_floats[:10]

['6900877', '6900876', '6902544', '6902545', '6902547', '6900799', '6900798', '6900797', '6900796', '4901043']


In [56]:
dac_substring = []
for index,row in gd_table.loc[:,['file']].iterrows():
    floatNum = row['file'].split('/')[1]
    all_floats.append(floatNum)

    if floatNum in good_floats:
        dac_substring.append(row['file'])
    

In [57]:
print dac_substring[:10]

['aoml/1900722/1900722_meta.nc', 'aoml/1901378/1901378_meta.nc', 'aoml/2900114/2900114_meta.nc', 'aoml/2900115/2900115_meta.nc', 'aoml/2900116/2900116_meta.nc', 'aoml/2900117/2900117_meta.nc', 'aoml/2900118/2900118_meta.nc', 'aoml/2900119/2900119_meta.nc', 'aoml/2900120/2900120_meta.nc', 'aoml/2900123/2900123_meta.nc']


In [67]:
tdsUrl = ''
for dac in dac_substring:
    tdsUrl += "http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/"
    tdsUrl += '/'.join(dac.split('/')[:2])
    tdsUrl += "/profiles/catalog.xml"
    print tdsUrl
    break

http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/aoml/1900722/profiles/catalog.xml


In [28]:
c = Crawl("http://thredds.aodn.org.au/thredds/catalog/IMOS/Argo/dac/coriolis/1900650/profiles/catalog.xml")
urls = [s.get("url") for d in c.datasets for s in d.services if s.get("service").lower() == "opendap"]

In [ ]:
def conv

In [32]:
floatLookup = {}
for url in urls:
    print url
    ds = Dataset(url)
    floatNum = url.split('/')[9]
    print floatNum
    p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], 
               ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])
    lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])
    
    # Convert o from micromol/kg to ml/l 
    ##perc_sat = 100 * convertTomll(o) / satO2(s,t) 
    print s,t,perc_sat
    floatLookup[floatNum] = (p,t,s,o,lat,lon,mtime)

http://thredds.aodn.org.au/thredds/dodsC/IMOS/Argo/dac/coriolis/1900650/profiles/D1900650_137.nc
1900650
34.8275 29.79

NameError: name 'perc_sat' is not defined

In [ ]:
floatLookup

In [33]:
p,t,s,o = (ds.variables['PRES_ADJUSTED'][0][0], ds.variables['TEMP_ADJUSTED'][0][0], ds.variables['PSAL_ADJUSTED'][0][0], ds.variables['DOXY_ADJUSTED'][0][0])

In [53]:
p,t,s,o

(4.3000002, 29.790001, 34.827457, 206.49001)

In [35]:
lat,lon,mtime = (ds.variables['LATITUDE'][0], ds.variables['LONGITUDE'][0], ds.variables['JULD'][0])

In [36]:
lat,lon,mtime

(6.0310000000000006, -14.026, 21985.069212962964)

In [118]:
#The p here is for depth...maybe
#That's why it said the unit was km
def dens_formula(p , t , s):
    C = (999.83 + (5.053*p) - (.048*(p**2))
    B =  .808 - (.0085*p)
    a = .0708*((1 + (.351*p)) + .068*(1 - (.0683*p))*t)
    y = .003*(1 - (.059*p) - .012*(1 - .064*p)*t)
    dens = C + (B*s) - (a*t) - ((y*(35 - s))*t)
    return dens

SyntaxError: invalid syntax (<ipython-input-118-819d3bb21dd0>, line 5)

In [115]:
dens_formula(p , t , s)

1039.220347679988

In [ ]:
def dens_formula(p , t , s):
    p = p / 10.0
    pw = 999.842594 + ((6.793952 * (10**-abs(2)))*t) - ((9.095290 * (10**-abs(3)))*(t**2)) + ((1.001685 * (10**-abs(4)))*(t**3)) - ((1.120083 * (10**-abs(6)))*(t**4)) + ((6.536332 * (10**-abs(9))*(t**5))        
    